In [54]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

#countryRoad = r'suc2cExudTip/none_1476_{}_10to14_20mn_0s_10/'
countryRoad =  r'getsrivssriset/{}_1476_{}_10to25_20mn_0s_64/'

In [55]:
# 1 vall per cell
nKonz = 8
volIdx=nKonz+1
coordId =volIdx+1 
# 1 vall per seg
lenIdx=coordId+1#volIdx+1
stIdx=lenIdx+1
orgIdId=stIdx+1
relLenId=orgIdId+1#segIdxid+1
segIdxid =relLenId+1

nToGet =segIdxid#elements + 3d coordinates of y , node+vol+len+st 

scenarios = ["earlyDry", "baseline", "lateDry"]
setIds = [5,44,61]
exceptPset = []#[('baseline','19'),('baseline','47'),('baseline','83')]
#result_list_compExcept = [path2file.format(scenario, str(setId)) for scenario, setId in exceptPsets]
result_list_comp = [(scenario, str(setId)) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

numPset = len(result_list_comp)
allPSets = [ i for i in range(numPset) if i not in exceptPset]

getAll = True

In [56]:

def getVal(mypathfilename,dtype=float,extension='.txt', header="infer",names = None):
    fullpath = pathresults +mypathfilename
    data_file_delimiter = ","
    if names is None:
        largest_column_count = 0
        with open(fullpath, 'r') as temp_f:
            lines = temp_f.readlines()
        last_line = lines[-1].strip()
        #print('last_line',last_line)
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        largest_column_count = len(array_from_last_line)
        names = [i for i in range(0, largest_column_count)]
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, 
                          header = header, names = names)
        #print('cst',cst.shape)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names)
    return cst

In [72]:

def get_last_line_as_numpy_array(file_path, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line

        try:
            last_line = lines[-1].strip()
        except:
            print('timeLine',file_path, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line.copy()

In [136]:
 
def wrapCellData(df, valname,time,gId, lens, sts,gIds, scenario, pipi):
    df = pd.DataFrame(df)
    lId = np.where(gId == gIds)
    df['time']=time[-df.shape[0]:]
    df['st']=sts[gId]
    df['lens']=lens.loc[:,gId].copy()[-df.shape[0]:]
    df['scenario']=scenario
    df['gId'] = gId
    df['pSet']=pipi
    df['vals']=valname
    return df

def getCellData(cid,gId,path2file, verbose = False):
    if verbose :
        print('getData_',cid,gId,path2file, volIdx,coordId)
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = getVal(path2file+fileName, dtype = float)
    if cid == volIdx:
        return cVol
    elif cid <= nKonz:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = getVal(path2file+fileName,
                                                 dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return cVol
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = getVal(path2file+fileName, dtype = float)
        konz = Q1/cVol #mol/cm3 or cm3/cm3
        return konz
    elif cid == coordId:
        fileName = "cyl_val/Cyl_coord_"+str(gId)+".txt"
        Q1 = getVal(path2file+fileName,
                                          dtype = float)

        return Q1
    raise Exception

In [58]:

GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
pSet = result_list_comp[0]
path2file = countryRoad.format(pSet[0],pSet[1])

In [59]:

time = np.array(list(
    pd.read_csv(pathresults + path2file + "time.txt", 
                   names = ["time","Qlight"])["time"][1:])) # because we have twice the initial value

rr = getVal(path2file+"rhizoSegsId.txt", dtype = int)

In [132]:

nKonz = 8
volIdx=nKonz+1
coordId =volIdx+1 
# 1 vall per seg
lenIdx=coordId+1#volIdx+1
stIdx=lenIdx+1
orgIdId=stIdx+1
relLenId=orgIdId+1#segIdxid+1
segIdxid =relLenId+1
valnames=  ['wat','cs','cl','coa',
            'cod','cca','ccd','css','co2',
            'vol','1dcoord']
lens = getVal(path2file+"length_Segs"+".txt", dtype =float)
sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float, 
                                   verbose = False)# last time steps 


Ginits = [pd.concat([wrapCellData(df=getCellData( cid, int( gId_),path2file), #a(df, time,gId, lens, sts)
                                 valname=valnames[cid],time=time,gId=int(gId_), lens=lens, sts=sts,
                        gIds=  rr.values[-1].tolist(),
                                scenario=pSet[0],pipi=pSet[1]) for gId_ in rr.values[-1].tolist()] , ignore_index=True)
                   for cid in range(coordId+1)]   # element[ idseg[ cells*time]]
# pd.concat(Ginitspd, ignore_index=True)


In [137]:
1+1

2

In [134]:
Ginitspd = pd.concat(Ginits, ignore_index=True)

In [135]:
Ginitspd 

,0,1,2,3,4,5,6,7,8,time,st,lens,scenario,gId,pipi,vals
0,0.000786,0.001155,0.001698,0.002497,0.003670,0.005395,0.007931,0.011659,0.017139,10.013889,0.0,1.0,earlyDry,0,5,wat
1,0.000786,0.001155,0.001698,0.002497,0.003670,0.005395,0.007931,0.011659,0.017139,10.027778,0.0,1.0,earlyDry,0,5,wat
2,0.000786,0.001155,0.001698,0.002497,0.003670,0.005395,0.007931,0.011659,0.017139,10.041667,0.0,1.0,earlyDry,0,5,wat
3,0.000786,0.001155,0.001698,0.002497,0.003670,0.005395,0.007931,0.011659,0.017139,10.055556,0.0,1.0,earlyDry,0,5,wat
4,0.000786,0.001155,0.001698,0.002497,0.003670,0.005395,0.007931,0.011659,0.017139,10.069444,0.0,1.0,earlyDry,0,5,wat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596930,0.024474,0.035424,0.051273,0.074212,0.107416,0.155474,0.225034,0.325715,0.471442,24.944444,1.0,NaN,earlyDry,514,5,1dcoord
2596931,0.024474,0.035424,0.051273,0.074212,0.107416,0.155474,0.225034,0.325715,0.471442,24.958333,1.0,NaN,earlyDry,514,5,1dcoord
2596932,0.024474,0.035424,0.051273,0.074212,0.107416,0.155474,0.225034,0.325715,0.471442,24.972222,1.0,NaN,earlyDry,514,5,1dcoord
2596933,0.024474,0.035424,0.051273,0.074212,0.107416,0.155474,0.225034,0.325715,0.471442,24.986111,1.0,NaN,earlyDry,514,5,1dcoord


In [121]:
merged_df = pd.concat(Ginitspd, ignore_index=True)

In [122]:
merged_df

,0
0,0 1 2 3 ...
1,0 1 2 3 ...
2,0 1 2 3 ...
3,0 1 2 3 ...
4,0 1 2 3 ...
...,...
5022,0 1 2 3 ...
5023,0 1 2 3 ...
5024,0 1 2 3 ...
5025,0 1 2 3 ...


In [ ]:

GiniAll = []
for pSet in range(result_list_comp):
    print(pSet)
    path2file = countryRoad.format(pSet[0],pSet[1])

    time = np.array(list(
        pd.read_csv(pathresults + path2file + "time.txt", 
                       names = ["time","Qlight"])["time"][1:])) # because we have twice the initial value

    rr = getVal(path2file+"rhizoSegsId.txt", dtype = int)

    lens = getVal(path2file+"length_Segs"+".txt", dtype =float)
    sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float, 
                                       verbose = False)# last time steps 


    Ginits = [[wrapCellData(df=getCellData( cid, int( gId_),path2file) #a(df, time,gId, lens, sts)
                                     ,time=time,gId=int(gId_), lens=lens, sts=sts,gIds=  rr.values[-1].tolist(),
                                    pSet[0],pSet[1]) for gId_ in rr.values[-1].tolist()] 
                       for cid in range(coordId+1)]# element[ idseg[ cells*time]]
    merged_df = pd.concat(dataframes, ignore_index=True)
    GiniAll.append(merged_df)